In [ ]:
import os
import json
import requests
from typing import List, Dict

In [ ]:
# -----------------------------
# LLM call
# -----------------------------
INFOMANIAK_API_URL = "https://api.infomaniak.com/ai/v1/chat/completions"
API_KEY = os.getenv("INFOMANIAK_API_KEY")  # À définir dans les variables d'environnement

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

MODEL = "llama3-8b"

# -----------------------------
# Fonction d'appel à Llama3 via Infomaniak
# -----------------------------
def evaluate_candidate(cahier_des_charges: str, cv_text: str, lm_text: str) -> Dict:
    prompt = f"""
Tu es un recruteur expert. Voici le cahier des charges du poste :
---
{cahier_des_charges}
---

Voici le CV du candidat :
---
{cv_text}
---

Et sa lettre de motivation :
---
{lm_text}
---

Évalue ce candidat sur une échelle de 0 à 100 en fonction de son adéquation au poste.
Fournis ta réponse AU FORMAT JSON UNIQUEMENT, avec les clés suivantes :
- "score": (entier entre 0 et 100)
- "justification": (une phrase concise expliquant le score)
- "points_forts": (liste de 2-3 points forts pertinents)
- "manques": (liste de 0 à 2 manques éventuels)

Ne donne aucun autre texte en dehors du JSON.
"""

    payload = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.3,
        "max_tokens": 500
    }

    try:
        response = requests.post(INFOMANIAK_API_URL, headers=HEADERS, json=payload)
        response.raise_for_status()
        result = response.json()
        content = result["choices"][0]["message"]["content"].strip()
        # Nettoyer les éventuelles balises de code
        if content.startswith("```json"):
            content = content[7:-3].strip()
        return json.loads(content)
    except Exception as e:
        print(f"Erreur lors de l'évaluation : {e}")
        return {"score": 0, "justification": "Erreur d'analyse", "points_forts": [], "manques": []}

In [ ]:

# -----------------------------
# Fonction principale
# -----------------------------
def trier_candidats(cahier_des_charges: str, candidats: List[Dict]) -> List[Dict]:
    """
    candidats = [
        {"nom": "Alice", "cv": "texte du CV...", "lm": "texte de la lettre..."},
        ...
    ]
    """
    resultats = []
    for candidat in candidats:
        print(f"Évaluation de {candidat['nom']}...")
        evaluation = evaluate_candidate(
            cahier_des_charges=cahier_des_charges,
            cv_text=candidat["cv"],
            lm_text=candidat["lm"]
        )
        resultats.append({
            "nom": candidat["nom"],
            "score": evaluation.get("score", 0),
            "justification": evaluation.get("justification", ""),
            "points_forts": evaluation.get("points_forts", []),
            "manques": evaluation.get("manques", [])
        })

    # Trier par score décroissant
    resultats.sort(key=lambda x: x["score"], reverse=True)
    return resultats



In [ ]:
# -----------------------------
# Exemple d'utilisation
# -----------------------------
if __name__ == "__main__":
    # Exemple de cahier des charges
    cahier = """
    Poste : Développeur Python senior
    Compétences requises :
    - 5 ans d'expérience en Python
    - Expérience avec FastAPI ou Django
    - Bonne connaissance des bases de données SQL et NoSQL
    - Capacité à travailler en équipe agile
    - Anglais professionnel
    """

    # Exemple de candidats (à remplacer par vos données)
    mes_candidats = [
        {
            "nom": "Jean Dupont",
            "cv": "Développeur Python depuis 6 ans. Expérience avec Django, PostgreSQL, MongoDB. Anglais courant.",
            "lm": "Passionné par les architectures robustes, j'ai mené plusieurs projets en équipe agile."
        },
        {
            "nom": "Marie Leroy",
            "cv": "Développeuse JavaScript, 4 ans d'expérience en React et Node.js.",
            "lm": "Je souhaite me reconvertir vers Python et apprendre rapidement."
        }
    ]

    classement = trier_candidats(cahier, mes_candidats)
    print("\n🏆 Classement des candidats :\n")
    for i, c in enumerate(classement, 1):
        print(f"{i}. {c['nom']} — Score : {c['score']}/100")
        print(f"   ➤ {c['justification']}")
        print(f"   ✅ Points forts : {', '.join(c['points_forts'])}")
        if c['manques']:
            print(f"   ⚠️  Manques : {', '.join(c['manques'])}")
        print()